In [19]:
# count number of files
import os
len(os.listdir('../books'))

8

In [20]:
!ls ../books

'1. Healt  Transformation Workshop (Subah Saraf and... (z-lib.org).pdf'
'1. Satvic Food Book  45 Healing Recipes to Cure An... (z-lib.org)(1).pdf'
'1. Satvic Food Book  45 Healing Recipes to Cure An... (z-lib.org)(2).pdf'
'1. Satvic Food Book  45 Healing Recipes to Cure An... (z-lib.org).epub'
'1. Satvic Food Book  45 Healing Recipes to Cure An... (z-lib.org).pdf'
'1. Satvic Movement - The Food Bookl (Subah Saraf) (z-lib.org).pdf'
'1. The Food Book - Hindi Edition (Subah Saraf) (z-lib.org).pdf'
'Subah Saraf - Satvic Movement.pdf'


In [21]:
# read 1st file
files_list = os.listdir('../books')
files_list.sort()
files_list

['1. Healt  Transformation Workshop (Subah Saraf and... (z-lib.org).pdf',
 '1. Satvic Food Book  45 Healing Recipes to Cure An... (z-lib.org)(1).pdf',
 '1. Satvic Food Book  45 Healing Recipes to Cure An... (z-lib.org)(2).pdf',
 '1. Satvic Food Book  45 Healing Recipes to Cure An... (z-lib.org).epub',
 '1. Satvic Food Book  45 Healing Recipes to Cure An... (z-lib.org).pdf',
 '1. Satvic Movement - The Food Bookl (Subah Saraf) (z-lib.org).pdf',
 '1. The Food Book - Hindi Edition (Subah Saraf) (z-lib.org).pdf',
 'Subah Saraf - Satvic Movement.pdf']

In [22]:
files_loc = '../books'
first_file = files_list[-1]
first_file

'Subah Saraf - Satvic Movement.pdf'

In [23]:
# read pdf file
import PyPDF2

# creating a pdf file object
pdfFileObj = open(files_loc + '/' + first_file, 'rb')

# creating a pdf reader object
pdfReader = PyPDF2.PdfReader(pdfFileObj)

In [24]:
# printing number of pages in pdf file
print(len(pdfReader.pages))

168


In [27]:
# creating a page object
pageObj = pdfReader.pages[0]

In [28]:
# extracting text from page
text = pageObj.extract_text()
text_split = text.split()
print(text)

# preference #1
if len(text_split) < 5:
  print('Text rejected1')
# preference equal #1
elif len([token for token in text_split if token.isdigit()]) > 21:
  print('Text rejected3')
elif 'index' in text.lower():
  print('Text rejected2')
else:
  # pass
  print(text)

# print(text.split())

THE  
FOOD 
BOOK
SUBAH SARAF

THE  
FOOD 
BOOK
SUBAH SARAF



In [ ]:
# convert pdf to text and pass it through ner model

In [29]:
import stanza
# stanza.install_corenlp()

In [30]:
import os
import re
from pathlib import Path

# Reimplement the logic to find the path where stanza_corenlp is installed.
core_nlp_path = os.getenv('CORENLP_HOME', str(Path.home() / 'stanza_corenlp'))

# A heuristic to find the right jar file
classpath = [str(p) for p in Path(core_nlp_path).iterdir() if re.match(r"stanford-corenlp-[0-9.]+\.jar", p.name)][0]
classpath

'/home/himani/stanza_corenlp/stanford-corenlp-4.5.5.jar'

In [31]:
from typing import List, Tuple, Generator
from tqdm.notebook import tqdm
from stanza.server import CoreNLPClient

def annotate_ner(ner_model_file: str, texts: List[str], tokenize_whitespace: bool = True):
    properties = {"ner.model": ner_model_file, "tokenize.whitespace": tokenize_whitespace, "ner.applyNumericClassifiers": False}
    
    annotated = []
    with CoreNLPClient(
         annotators=['tokenize','ssplit','ner'],
         properties=properties,
         timeout=30000,
         be_quiet=True,
        memory='6G') as client:
    
        for text in tqdm(texts):
            annotated.append(client.annotate(text))
    return annotated

In [32]:
# load model
model_name = 'ar'
model_file = f'{model_name}.model.ser.gz'
ner_prop_filename = f'{model_name}.model.props'

In [33]:
annotations = annotate_ner(model_file,
                           [text])

2023-09-22 06:18:00 INFO: Writing properties to tmp file: corenlp_server-85e1da8465ed4d55.props
2023-09-22 06:18:00 INFO: Starting server with command: java -Xmx6G -cp /home/himani/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-85e1da8465ed4d55.props -annotators tokenize,ssplit,ner -preload -outputFormat serialized


  0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
annotations[0].sentence[0].token[3]

word: "SUBAH"
pos: "NNP"
value: "SUBAH"
originalText: "SUBAH"
ner: "NAME"
coarseNER: "NAME"
fineGrainedNER: "NAME"
nerLabelProbs: "NAME=0.7447941399422754"
lemma: "SUBAH"
beginChar: 17
endChar: 22
tokenBeginIndex: 3
tokenEndIndex: 4
hasXmlContext: false
isNewline: false
entityMentionIndex: 1

In [36]:
from dataclasses import dataclass, asdict
import pandas as pd

@dataclass
class NERData:
    ner: List[str]
    tokens: List[str]
        
    # Let's use Pandas to make it pretty in a notebook
    def _repr_html_(self):
        return pd.DataFrame(asdict(self)).T._repr_html_()

def extract_ner_data(annotation) -> NERData:
    tokens = [token for sentence in annotation.sentence for token in sentence.token]
    return NERData(tokens=[t.word for t in tokens], ner=[t.coarseNER for t in tokens])

In [37]:
extract_ner_data(annotations[0])

,0,1,2,3,4
ner,QUANTITY,NAME,NAME,NAME,NAME
tokens,THE,FOOD,BOOK,SUBAH,SARAF
